# **Phase 2 (Group 29) - Data Staging**
Émilie Brazeau (300209120), Nicholas Gin (300107597), Gordon Tang (300132939)

Link to Google Colab: https://colab.research.google.com/drive/1uW1q-00w0KnWrTeEMgB16Uaf-1XaOG_X?usp=sharing

In [ ]:
import pandas as pd
from google.colab import files

import nltk
nltk.download('stopwords')
nltk.download("wordnet")
nltk.download("omw-1.4")
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

import re

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


# **Disneyland Reviews Dataset**
# Extract

In [ ]:
# Loading the data
url='https://drive.google.com/file/d/1atEjlCz6cNs4_bKLd2Mwe-W-MIKwz3AU/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df_dis = pd.read_csv(url, encoding='cp1252')

df_dis.head(9)

,Review_ID,Rating,Year_Month,Reviewer_Location,Review_Text,Branch
0,670772142,4,2019-4,Australia,If you've ever been to Disneyland anywhere you...,Disneyland_HongKong
1,670682799,4,2019-5,Philippines,Its been a while since d last time we visit HK...,Disneyland_HongKong
2,670623270,4,2019-4,United Arab Emirates,Thanks God it wasn t too hot or too humid wh...,Disneyland_HongKong
3,670607911,4,2019-4,Australia,HK Disneyland is a great compact park. Unfortu...,Disneyland_HongKong
4,670607296,4,2019-4,United Kingdom,"the location is not in the city, took around 1...",Disneyland_HongKong
5,670591897,3,2019-4,Singapore,"Have been to Disney World, Disneyland Anaheim ...",Disneyland_HongKong
6,670585330,5,2019-4,India,Great place! Your day will go by and you won't...,Disneyland_HongKong
7,670574142,3,2019-3,Malaysia,Think of it as an intro to Disney magic for th...,Disneyland_HongKong
8,670571027,2,2019-4,Australia,"Feel so let down with this place,the Disneylan...",Disneyland_HongKong


In [ ]:
print(df_dis.dtypes)

Review_ID             int64
Rating                int64
Year_Month           object
Reviewer_Location    object
Review_Text          object
Branch               object
dtype: object


# Transform

**Clean up data**

In [ ]:
# Strip whitespaces and extra blank spaces in the Review Texts.
def strip(text):
    # Remove any leading/trailing whitespace
    text = text.strip()
    # Replace any runs of whitespace with a single space
    text = re.sub(r'\s+', ' ', text)
    # Add a space after any punctuation marks (except apostrophes or if already separated by whitespace)
    # Do this for consistency
    text = re.sub(r'(?<=[^\s\w])|(?=[^\s\w])', ' ', text)
    # Remove any double spaces
    text = re.sub(r'\s\s+', ' ', text)
    return text

# Apply the strip function to the 'Review_Text' column
df_dis['Review_Text'] = df_dis['Review_Text'].apply(strip)
# Normalizing the Review Texts (decapitalizing the Review Texts).
df_dis['Review_Text'] = df_dis['Review_Text'].str.lower()
# Remove unicode characters.
pattern = r'[^a-z0-9\s]'
df_dis['Review_Text'] = df_dis['Review_Text'].str.replace(pattern, '', regex=True)
df_dis['Review_Text'] = df_dis['Review_Text'].str.replace(r"(@\[\w+\])|(#\w+)|(\w+:\/\/\S+)|^rt|http.+?", "", regex=True)
# Remove stop words.
stop_words = stopwords.words('english')
df_dis['Review_Text'] = df_dis['Review_Text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
# Lemmatize words.
lemmatizer = WordNetLemmatizer()
df_dis['Review_Text'] = df_dis['Review_Text'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))

# For formatting consistency (with the Universal Studios Reviews dataset), update the formatting of each Disneyland branch.
df_dis['Branch'] = df_dis['Branch'].replace('Disneyland_HongKong', 'Disneyland Hong Kong').replace('Disneyland_Paris', 'Disneyland Paris').replace('Disneyland_California', 'Disneyland California')

df_dis.head(20)

,Review_ID,Rating,Year_Month,Reviewer_Location,Review_Text,Branch
0,670772142,4,2019-4,Australia,ever disneyland anywhere find disneyland hong ...,Disneyland Hong Kong
1,670682799,4,2019-5,Philippines,since last time visit hk disneyland yet time s...,Disneyland Hong Kong
2,670623270,4,2019-4,United Arab Emirates,thanks god hot humid visiting park otherwise w...,Disneyland Hong Kong
3,670607911,4,2019-4,Australia,hk disneyland great compact park unfortunately...,Disneyland Hong Kong
4,670607296,4,2019-4,United Kingdom,location city took around 1 hour kowlon kid li...,Disneyland Hong Kong
5,670591897,3,2019-4,Singapore,disney world disneyland anaheim tokyo disneyla...,Disneyland Hong Kong
6,670585330,5,2019-4,India,great place day go even know obviously went da...,Disneyland Hong Kong
7,670574142,3,2019-3,Malaysia,think intro disney magic little one almost att...,Disneyland Hong Kong
8,670571027,2,2019-4,Australia,feel let place disneyland train fantastic get ...,Disneyland Hong Kong
9,670570869,5,2019-3,India,go talking disneyland whatever say le disneyla...,Disneyland Hong Kong


**Drop unrequired columns, remove null values and split columns where necessary**

In [ ]:
# Split the 'Year_Month' column into two separate columns for the 'Year' and 'Month', respectively.
# Values in the 'Year_Month' column are in one of two formats: yyyy-mm or yyyy-m (with yyyy representing a four digit year,
# mm representing a two digit month, m representing a one digit month).
df_dis[['Year', 'Month']] = df_dis['Year_Month'].str.split('-', expand=True)

# Replace all 'missing' values (note these are not null values) in the 'Year' column with -1.
df_dis['Year'] = df_dis['Year'].replace('missing', '-1')
# Set the 'Year' column type to int.
df_dis['Year'] = df_dis['Year'].astype(int)

# Replace all null values in the 'Month' column with -1.
df_dis['Month'] = df_dis['Month'].fillna(-1)
# Set the 'Month' column type to int.
df_dis['Month'] = df_dis['Month'].astype(int)

# Remove any entries in the dataset with null values (we are not able to add this information from other datasets).
df_dis = df_dis[df_dis['Year'] != -1]
df_dis = df_dis[df_dis['Month'] != -1]

# Create 'Quarter' column.
df_dis['Quarter'] = pd.to_datetime(df_dis['Month'], format='%m').dt.quarter

# Remove rows with empty Review_Texts.
df_dis = df_dis[df_dis['Review_Text'] != '']

# Drop columns that are not found in the Universal Studios Reviews dataset, as well as the 'Year_Month' column, since it is no longer required.
df_dis.drop(columns = ['Year_Month', 'Reviewer_Location', 'Review_ID'])

# Re-order the columns in the dataset.
df_dis = df_dis.reindex(columns=['Review_Text', 'Rating', 'Month', 'Quarter', 'Year', 'Branch'])

df_dis.head(30)

,Review_Text,Rating,Month,Quarter,Year,Branch
0,ever disneyland anywhere find disneyland hong ...,4,4,2,2019,Disneyland Hong Kong
1,since last time visit hk disneyland yet time s...,4,5,2,2019,Disneyland Hong Kong
2,thanks god hot humid visiting park otherwise w...,4,4,2,2019,Disneyland Hong Kong
3,hk disneyland great compact park unfortunately...,4,4,2,2019,Disneyland Hong Kong
4,location city took around 1 hour kowlon kid li...,4,4,2,2019,Disneyland Hong Kong
5,disney world disneyland anaheim tokyo disneyla...,3,4,2,2019,Disneyland Hong Kong
6,great place day go even know obviously went da...,5,4,2,2019,Disneyland Hong Kong
7,think intro disney magic little one almost att...,3,3,1,2019,Disneyland Hong Kong
8,feel let place disneyland train fantastic get ...,2,4,2,2019,Disneyland Hong Kong
9,go talking disneyland whatever say le disneyla...,5,3,1,2019,Disneyland Hong Kong


**Remove noisy data and duplicates**

In [ ]:
df_dis = df_dis[df_dis['Review_Text'] != "activex vt error"] # Noisy data
df_dis = df_dis.drop_duplicates(keep = 'first') # Drop duplicates

**Generate measures/facts**

In [ ]:
# Define a function to calculate the percentage of ratings in each group
def rating_percentages(group):
    # Calculate the percentage of positive ratings by counting the number of ratings that are 4 or 5, and dividing by the total number of ratings in the group
    positive = group[group['Rating'] >= 4]['Rating'].count() / group['Rating'].count()
    # Calculate the percentage of mixed ratings by counting the number of ratings that are 3, and dividing by the total number of ratings in the group
    mixed = group[group['Rating'] == 3]['Rating'].count() / group['Rating'].count()
    # Calculate the percentage of negative ratings by counting the number of ratings that are 1 or 2, and dividing by the total number of ratings in the group
    negative = group[group['Rating'] <= 2]['Rating'].count() / group['Rating'].count()
    # Return a Pandas Series object with the calculated percentages as values and the column names as the index
    return pd.Series({'Monthly % of Positive Reviews': positive * 100, 'Monthly % of Mixed Reviews': mixed * 100, 'Monthly % of Negative Reviews': negative * 100})

percentages_dis = df_dis.groupby(['Branch', 'Year', 'Month']).apply(rating_percentages)

percentages_dis

Monthly % of Positive Reviews  \
Branch                Year Month                                  
Disneyland California 2010 4                         100.000000   
                           5                         100.000000   
                           6                          60.000000   
                           7                          75.000000   
                           8                          80.000000   
...                                                         ...   
Disneyland Paris      2018 12                         61.206897   
                      2019 1                          73.770492   
                           2                          74.576271   
                           3                          61.764706   
                           4                          52.941176   

                                  Monthly % of Mixed Reviews  \
Branch                Year Month                               
Disneyland California 2010 4                        0.000000   
                           5                        0.000000   
                           6                       20.000000   
                           7                       25.000000   
                           8                        0.000000   
...                                                      ...   
Disneyland Paris      2018 12                      17.241379   
                      2019 1                       11.475410   
                           2                       13.559322   
                           3                       16.176471   
                           4                       22.058824   

                                  Monthly % of Negative Reviews  
Branch                Year Month                                 
Disneyland California 2010 4                           0.000000  
                           5                           0.000000  
                           6                          20.000000  
                           7                           0.000000  
                           8                          20.000000  
...                                                         ...  
Disneyland Paris      2018 12                         21.551724  
                      2019 1                          14.754098  
                           2                          11.864407  
                           3                          22.058824  
                           4                          25.000000  

[326 rows x 3 columns]

In [ ]:
df_dis = df_dis.merge(percentages_dis, how='outer', on=['Branch', 'Year', 'Month'])

df_dis.head(300)

,Review_Text,Rating,Month,Quarter,Year,Branch,Monthly % of Positive Reviews,Monthly % of Mixed Reviews,Monthly % of Negative Reviews
0,ever disneyland anywhere find disneyland hong ...,4,4,2,2019,Disneyland Hong Kong,78.688525,18.032787,3.278689
1,thanks god hot humid visiting park otherwise w...,4,4,2,2019,Disneyland Hong Kong,78.688525,18.032787,3.278689
2,hk disneyland great compact park unfortunately...,4,4,2,2019,Disneyland Hong Kong,78.688525,18.032787,3.278689
3,location city took around 1 hour kowlon kid li...,4,4,2,2019,Disneyland Hong Kong,78.688525,18.032787,3.278689
4,disney world disneyland anaheim tokyo disneyla...,3,4,2,2019,Disneyland Hong Kong,78.688525,18.032787,3.278689
...,...,...,...,...,...,...,...,...,...
295,every disney park world one may favorite 4 vis...,5,11,4,2018,Disneyland Hong Kong,75.806452,14.516129,9.677419
296,fl check sadly think better spending second da...,3,11,4,2018,Disneyland Hong Kong,75.806452,14.516129,9.677419
297,simply best day hong kong disney read negative...,5,11,4,2018,Disneyland Hong Kong,75.806452,14.516129,9.677419
298,went week day would recommend even hundred peo...,4,11,4,2018,Disneyland Hong Kong,75.806452,14.516129,9.677419


In [ ]:
print(df_dis.isnull().sum())

Review_Text                      0
Rating                           0
Month                            0
Quarter                          0
Year                             0
Branch                           0
Monthly % of Positive Reviews    0
Monthly % of Mixed Reviews       0
Monthly % of Negative Reviews    0
dtype: int64


# **Universal Studios Reviews Dataset**
# Extract

In [ ]:
# Loading the data
url='https://drive.google.com/file/d/1Avu5OYvmN8kWFzICK3_KE13Ob1V0uPwP/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df_uni = pd.read_csv(url, encoding='UTF-8')

df_uni

,reviewer,rating,written_date,title,review_text,branch
0,Kelly B,2.0,"May 30, 2021",Universal is a complete Disaster - stick with ...,We went to Universal over Memorial Day weekend...,Universal Studios Florida
1,Jon,1.0,"May 30, 2021",Food is hard to get.,The food service is horrible. I’m not reviewin...,Universal Studios Florida
2,Nerdy P,2.0,"May 30, 2021",Disappointed,I booked this vacation mainly to ride Hagrid m...,Universal Studios Florida
3,ran101278,4.0,"May 29, 2021",My opinion,When a person tries the test seat for the ride...,Universal Studios Florida
4,tammies20132015,5.0,"May 28, 2021",The Bourne Stuntacular...MUST SEE,"Ok, I can't stress enough to anyone and everyo...",Universal Studios Florida
...,...,...,...,...,...,...
50899,vinz20,4.0,"March 29, 2010",I'll Be Back Only If ...,This is my first visit to a Universal Studio t...,Universal Studios Singapore
50900,betty l,4.0,"March 29, 2010",Universal Studios Singapore Experience,We finally visited Singapore's very first them...,Universal Studios Singapore
50901,spoonos65,4.0,"March 28, 2010",Impressive but not quite finished!,We visited during the first week of its 'soft ...,Universal Studios Singapore
50902,HeatSeekerWrexham_UK,4.0,"March 22, 2010",Small but beautifully marked,We visited on the 3rd day of the 'soft' openin...,Universal Studios Singapore


In [ ]:
print(df_uni.dtypes)

reviewer         object
rating          float64
written_date     object
title            object
review_text      object
branch           object
dtype: object


# Transform

**Clean up data**

In [ ]:
# Clean up all entries in the review_text column.

# Strip whitespaces and extra blank space in the review texts.
df_uni['review_text'] = df_uni['review_text'].apply(strip)
# Normalizing the Review Texts (decapitalizing the Review Texts).
df_uni['review_text'] = df_uni['review_text'].str.lower()
# Remove unicode characters.
df_uni['review_text'] = df_uni['review_text'].str.replace(pattern, '', regex=True)
df_uni['review_text'] = df_uni['review_text'].str.replace(r"(@\[\w+\])|(#\w+)|(\w+:\/\/\S+)|^rt|http.+?", "", regex=True)
# Remove stop words.
df_uni['review_text'] = df_uni['review_text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
# Stem words.
df_uni['review_text'] = df_uni['review_text'].apply(lambda x: ' '.join([lemmatizer.lemmatize(word) for word in x.split()]))

# Set the ratings in the 'rating' column to type int to be consistent with the Disneyland Reviews dataset.
df_uni['rating'] = df_uni['rating'].astype(int)

# Reformat column names in the Universal Studios Reviews dataset to match the columns in the Disneyland Reviews dataset.
# Note that we are only reformatting columns we will be keeping; we will be dropping columns we don't need in the next step.
df_uni = df_uni.rename({'rating': 'Rating', 'review_text': 'Review_Text', 'branch': 'Branch'}, axis=1)

df_uni.head(20)

,reviewer,Rating,written_date,title,Review_Text,Branch
0,Kelly B,2,"May 30, 2021",Universal is a complete Disaster - stick with ...,went universal memorial day weekend total trai...,Universal Studios Florida
1,Jon,1,"May 30, 2021",Food is hard to get.,food service horrible reviewing food wait time...,Universal Studios Florida
2,Nerdy P,2,"May 30, 2021",Disappointed,booked vacation mainly ride hagrid motorcycle ...,Universal Studios Florida
3,ran101278,4,"May 29, 2021",My opinion,person try test seat ride get green light go l...,Universal Studios Florida
4,tammies20132015,5,"May 28, 2021",The Bourne Stuntacular...MUST SEE,ok stress enough anyone everyone go universal ...,Universal Studios Florida
5,John,1,"May 28, 2021",This is not a vacation,worst experience ever ride outdated whole plac...,Universal Studios Florida
6,annapN7702ZW,2,"May 27, 2021",Expected More,expected alot waiting around lack staffing pri...,Universal Studios Florida
7,Deb P,2,"May 27, 2021",Disapointing.....,4th trip daughter universal unfortunately disa...,Universal Studios Florida
8,Chuck N,1,"May 27, 2021",Greed makes for a terrible guest experience,universal one thing disney everything disney w...,Universal Studios Florida
9,Jen,4,"May 26, 2021",Good first time visit with kids,spent 6 night site sapphire fall family 6 2 ad...,Universal Studios Florida


**Drop unrequired columns and split columns where necessary**

In [ ]:
# To be consistent with the Disneyland Reviews dataset, split the 'written_date' column into two separate columns for the 'Year' and 'Month', respectively.
# We will not be saving the day from the date.
df_uni['written_date'] = pd.to_datetime(df_uni['written_date'])
df_uni['Month'] = df_uni['written_date'].dt.month
df_uni['Year'] = df_uni['written_date'].dt.year

# Create 'Quarter' column.
df_uni['Quarter'] = pd.to_datetime(df_uni['Month'], format='%m').dt.quarter

# Remove rows with empty Review_Texts.
df_uni = df_uni[df_uni['Review_Text'] != '']

# Drop columns that are not found in the Disneyland Reviews dataset, as well as the 'written_date' column, since it is no longer required.
df_uni = df_uni.drop(columns = ['reviewer', 'written_date', 'title'])

# Re-order the columns in the dataset.
df_uni = df_uni.reindex(columns=['Review_Text', 'Rating', 'Month', 'Quarter', 'Year', 'Branch'])

df_uni.head(30)

,Review_Text,Rating,Month,Quarter,Year,Branch
0,went universal memorial day weekend total trai...,2,5,2,2021,Universal Studios Florida
1,food service horrible reviewing food wait time...,1,5,2,2021,Universal Studios Florida
2,booked vacation mainly ride hagrid motorcycle ...,2,5,2,2021,Universal Studios Florida
3,person try test seat ride get green light go l...,4,5,2,2021,Universal Studios Florida
4,ok stress enough anyone everyone go universal ...,5,5,2,2021,Universal Studios Florida
5,worst experience ever ride outdated whole plac...,1,5,2,2021,Universal Studios Florida
6,expected alot waiting around lack staffing pri...,2,5,2,2021,Universal Studios Florida
7,4th trip daughter universal unfortunately disa...,2,5,2,2021,Universal Studios Florida
8,universal one thing disney everything disney w...,1,5,2,2021,Universal Studios Florida
9,spent 6 night site sapphire fall family 6 2 ad...,4,5,2,2021,Universal Studios Florida


**Remove null values, noisy data and duplicates**

In [ ]:
df_uni = df_uni.drop_duplicates(keep = 'first') # Drop duplicates

# Checking for null values in the Universal Studios dataset
print(df_uni.isnull().sum())

Review_Text    0
Rating         0
Month          0
Quarter        0
Year           0
Branch         0
dtype: int64


**Generate measures/facts**

In [ ]:
percentages_uni = df_uni.groupby(['Branch', 'Year', 'Month']).apply(rating_percentages)

percentages_uni

Monthly % of Positive Reviews  \
Branch                      Year Month                                  
Universal Studios Florida   2002 10                          0.000000   
                            2003 3                         100.000000   
                                 4                         100.000000   
                                 6                         100.000000   
                                 7                         100.000000   
...                                                               ...   
Universal Studios Singapore 2021 1                          33.333333   
                                 2                          71.428571   
                                 3                          66.666667   
                                 4                          71.428571   
                                 5                         100.000000   

                                        Monthly % of Mixed Reviews  \
Branch                      Year Month                               
Universal Studios Florida   2002 10                       0.000000   
                            2003 3                        0.000000   
                                 4                        0.000000   
                                 6                        0.000000   
                                 7                        0.000000   
...                                                            ...   
Universal Studios Singapore 2021 1                       16.666667   
                                 2                       28.571429   
                                 3                        0.000000   
                                 4                        0.000000   
                                 5                        0.000000   

                                        Monthly % of Negative Reviews  
Branch                      Year Month                                 
Universal Studios Florida   2002 10                        100.000000  
                            2003 3                           0.000000  
                                 4                           0.000000  
                                 6                           0.000000  
                                 7                           0.000000  
...                                                               ...  
Universal Studios Singapore 2021 1                          50.000000  
                                 2                           0.000000  
                                 3                          33.333333  
                                 4                          28.571429  
                                 5                           0.000000  

[477 rows x 3 columns]

In [ ]:
df_uni = df_uni.merge(percentages_uni, how='outer', on=['Branch', 'Year', 'Month'])

df_uni.head(300)

,Review_Text,Rating,Month,Quarter,Year,Branch,Monthly % of Positive Reviews,Monthly % of Mixed Reviews,Monthly % of Negative Reviews
0,went universal memorial day weekend total trai...,2,5,2,2021,Universal Studios Florida,36.734694,8.163265,55.102041
1,food service horrible reviewing food wait time...,1,5,2,2021,Universal Studios Florida,36.734694,8.163265,55.102041
2,booked vacation mainly ride hagrid motorcycle ...,2,5,2,2021,Universal Studios Florida,36.734694,8.163265,55.102041
3,person try test seat ride get green light go l...,4,5,2,2021,Universal Studios Florida,36.734694,8.163265,55.102041
4,ok stress enough anyone everyone go universal ...,5,5,2,2021,Universal Studios Florida,36.734694,8.163265,55.102041
...,...,...,...,...,...,...,...,...,...
295,purchased meal deal citywalk went margaritavil...,1,12,4,2020,Universal Studios Florida,42.222222,11.111111,46.666667
296,universal amazing job pandemic park clean ever...,5,12,4,2020,Universal Studios Florida,42.222222,11.111111,46.666667
297,absolutely adore place universal park definite...,5,12,4,2020,Universal Studios Florida,42.222222,11.111111,46.666667
298,family 5 two adult 8 6 6 750 spent one day 657...,2,12,4,2020,Universal Studios Florida,42.222222,11.111111,46.666667


In [ ]:
# Check for null values
print(df_uni.isnull().sum())

Review_Text                      0
Rating                           0
Month                            0
Quarter                          0
Year                             0
Branch                           0
Monthly % of Positive Reviews    0
Monthly % of Mixed Reviews       0
Monthly % of Negative Reviews    0
dtype: int64


# Dataset Integration (Enrich Disneyland Reviews with Universal Studios Reviews)

In [ ]:
# Merge the Disneyland Reviews and Universal Studios Reviews datasets.
df_reviews = pd.merge(df_dis, df_uni, how='outer', on=['Rating', 'Review_Text', 'Month', 'Quarter', 'Year', 'Branch', 'Monthly % of Positive Reviews', 'Monthly % of Mixed Reviews', 'Monthly % of Negative Reviews'])
df_reviews

,Review_Text,Rating,Month,Quarter,Year,Branch,Monthly % of Positive Reviews,Monthly % of Mixed Reviews,Monthly % of Negative Reviews
0,ever disneyland anywhere find disneyland hong ...,4,4,2,2019,Disneyland Hong Kong,78.688525,18.032787,3.278689
1,thanks god hot humid visiting park otherwise w...,4,4,2,2019,Disneyland Hong Kong,78.688525,18.032787,3.278689
2,hk disneyland great compact park unfortunately...,4,4,2,2019,Disneyland Hong Kong,78.688525,18.032787,3.278689
3,location city took around 1 hour kowlon kid li...,4,4,2,2019,Disneyland Hong Kong,78.688525,18.032787,3.278689
4,disney world disneyland anaheim tokyo disneyla...,3,4,2,2019,Disneyland Hong Kong,78.688525,18.032787,3.278689
...,...,...,...,...,...,...,...,...,...
90875,first visit universal studio theme park went p...,4,3,1,2010,Universal Studios Singapore,100.000000,0.000000,0.000000
90876,finally visited singapore first theme park uni...,4,3,1,2010,Universal Studios Singapore,100.000000,0.000000,0.000000
90877,visited first week soft opening unfortunately ...,4,3,1,2010,Universal Studios Singapore,100.000000,0.000000,0.000000
90878,visited 3rd day soft opening ticket sale limit...,4,3,1,2010,Universal Studios Singapore,100.000000,0.000000,0.000000


In [ ]:
# Check for null values after the mereg
print(df_reviews.isnull().sum())

Review_Text                      0
Rating                           0
Month                            0
Quarter                          0
Year                             0
Branch                           0
Monthly % of Positive Reviews    0
Monthly % of Mixed Reviews       0
Monthly % of Negative Reviews    0
dtype: int64


# **Park Attendance Dataset**
# Extract

In [ ]:
# Loading the data 
# (We manually compiled this data ourselves with the goal of using the data
# for data for this project, so it is imported clean).
url='https://drive.google.com/file/d/1m3_DJbGPaYLTtgtfVB0k9Mz5GY23toAH/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df_att = pd.read_csv(url, encoding='UTF-8')

df_att

,Branch,2006 Attendance (millions),2007 Attendance (millions),2008 Attendance (millions),2009 Attendance (millions),2010 Attendance (millions),2011 Attendance (millions),2012 Attendance (millions),2013 Attendance (millions),2014 Attendance (millions),2015 Attendance (millions),2016 Attendance (millions),2017 Attendance (millions),2018 Attendance (millions),2019 Attendance (millions),2020 Attendance (millions),2021 Attendance (millions)
0,Disneyland California,14.73,14.870,14.720,15.90,15.980,16.140,15.960,16.200,16.770,18.280,17.940,18.300,18.660,18.660,3.670,8.500
1,Disneyland Paris,10.60,12.000,12.688,12.74,10.500,10.990,11.200,10.430,9.940,10.360,8.400,9.660,9.843,9.745,2.620,3.500
2,Disneyland Hong Kong,5.20,4.150,4.500,4.60,5.200,5.900,6.700,7.400,7.600,6.800,6.100,6.200,6.700,5.695,1.700,2.600
3,Universal Studios Japan,8.50,8.713,8.300,8.00,8.160,8.500,9.700,10.100,11.800,13.900,14.500,14.935,14.300,14.500,4.901,5.500
4,Universal Studios Florida,6.00,6.200,6.231,5.53,5.925,6.044,6.195,7.062,8.263,9.585,9.998,10.198,10.708,10.922,4.096,8.987
5,Universal Studios Singapore,NaN,NaN,NaN,NaN,2.000,3.411,3.480,3.650,3.840,4.200,4.100,4.220,4.400,4.500,1.098,1.200


# Dataset Integration (Enrich reviews with attendance information)

In [ ]:
# Merge the attendance data into the review dataframe based on the branch and year
df_with_att = pd.merge(df_reviews, df_att, how='outer', on=['Branch'])

df_with_att

,Review_Text,Rating,Month,Quarter,Year,Branch,Monthly % of Positive Reviews,Monthly % of Mixed Reviews,Monthly % of Negative Reviews,2006 Attendance (millions),...,2012 Attendance (millions),2013 Attendance (millions),2014 Attendance (millions),2015 Attendance (millions),2016 Attendance (millions),2017 Attendance (millions),2018 Attendance (millions),2019 Attendance (millions),2020 Attendance (millions),2021 Attendance (millions)
0,ever disneyland anywhere find disneyland hong ...,4,4,2,2019,Disneyland Hong Kong,78.688525,18.032787,3.278689,5.2,...,6.70,7.40,7.60,6.8,6.1,6.20,6.7,5.695,1.700,2.6
1,thanks god hot humid visiting park otherwise w...,4,4,2,2019,Disneyland Hong Kong,78.688525,18.032787,3.278689,5.2,...,6.70,7.40,7.60,6.8,6.1,6.20,6.7,5.695,1.700,2.6
2,hk disneyland great compact park unfortunately...,4,4,2,2019,Disneyland Hong Kong,78.688525,18.032787,3.278689,5.2,...,6.70,7.40,7.60,6.8,6.1,6.20,6.7,5.695,1.700,2.6
3,location city took around 1 hour kowlon kid li...,4,4,2,2019,Disneyland Hong Kong,78.688525,18.032787,3.278689,5.2,...,6.70,7.40,7.60,6.8,6.1,6.20,6.7,5.695,1.700,2.6
4,disney world disneyland anaheim tokyo disneyla...,3,4,2,2019,Disneyland Hong Kong,78.688525,18.032787,3.278689,5.2,...,6.70,7.40,7.60,6.8,6.1,6.20,6.7,5.695,1.700,2.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90875,first visit universal studio theme park went p...,4,3,1,2010,Universal Studios Singapore,100.000000,0.000000,0.000000,NaN,...,3.48,3.65,3.84,4.2,4.1,4.22,4.4,4.500,1.098,1.2
90876,finally visited singapore first theme park uni...,4,3,1,2010,Universal Studios Singapore,100.000000,0.000000,0.000000,NaN,...,3.48,3.65,3.84,4.2,4.1,4.22,4.4,4.500,1.098,1.2
90877,visited first week soft opening unfortunately ...,4,3,1,2010,Universal Studios Singapore,100.000000,0.000000,0.000000,NaN,...,3.48,3.65,3.84,4.2,4.1,4.22,4.4,4.500,1.098,1.2
90878,visited 3rd day soft opening ticket sale limit...,4,3,1,2010,Universal Studios Singapore,100.000000,0.000000,0.000000,NaN,...,3.48,3.65,3.84,4.2,4.1,4.22,4.4,4.500,1.098,1.2


In [ ]:
# Remove reviews that have the same Review Text.
df_with_att = df_with_att.drop_duplicates(subset = ['Review_Text'], keep = False)

# Generate Surrogate Keys

In [ ]:
# Generate surrogate keys
df_with_att = df_with_att.reset_index(drop=True) # Reset index
df_with_att['Surrogate Key'] = range(1,len(df_with_att)+1)
df = df_with_att.reindex(columns=['Surrogate Key'] + list([c for c in df_with_att.columns if c!= 'Surrogate Key']))

df

,Surrogate Key,Review_Text,Rating,Month,Quarter,Year,Branch,Monthly % of Positive Reviews,Monthly % of Mixed Reviews,Monthly % of Negative Reviews,...,2012 Attendance (millions),2013 Attendance (millions),2014 Attendance (millions),2015 Attendance (millions),2016 Attendance (millions),2017 Attendance (millions),2018 Attendance (millions),2019 Attendance (millions),2020 Attendance (millions),2021 Attendance (millions)
0,1,ever disneyland anywhere find disneyland hong ...,4,4,2,2019,Disneyland Hong Kong,78.688525,18.032787,3.278689,...,6.70,7.40,7.60,6.8,6.1,6.20,6.7,5.695,1.700,2.6
1,2,thanks god hot humid visiting park otherwise w...,4,4,2,2019,Disneyland Hong Kong,78.688525,18.032787,3.278689,...,6.70,7.40,7.60,6.8,6.1,6.20,6.7,5.695,1.700,2.6
2,3,hk disneyland great compact park unfortunately...,4,4,2,2019,Disneyland Hong Kong,78.688525,18.032787,3.278689,...,6.70,7.40,7.60,6.8,6.1,6.20,6.7,5.695,1.700,2.6
3,4,location city took around 1 hour kowlon kid li...,4,4,2,2019,Disneyland Hong Kong,78.688525,18.032787,3.278689,...,6.70,7.40,7.60,6.8,6.1,6.20,6.7,5.695,1.700,2.6
4,5,disney world disneyland anaheim tokyo disneyla...,3,4,2,2019,Disneyland Hong Kong,78.688525,18.032787,3.278689,...,6.70,7.40,7.60,6.8,6.1,6.20,6.7,5.695,1.700,2.6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90841,90842,first visit universal studio theme park went p...,4,3,1,2010,Universal Studios Singapore,100.000000,0.000000,0.000000,...,3.48,3.65,3.84,4.2,4.1,4.22,4.4,4.500,1.098,1.2
90842,90843,finally visited singapore first theme park uni...,4,3,1,2010,Universal Studios Singapore,100.000000,0.000000,0.000000,...,3.48,3.65,3.84,4.2,4.1,4.22,4.4,4.500,1.098,1.2
90843,90844,visited first week soft opening unfortunately ...,4,3,1,2010,Universal Studios Singapore,100.000000,0.000000,0.000000,...,3.48,3.65,3.84,4.2,4.1,4.22,4.4,4.500,1.098,1.2
90844,90845,visited 3rd day soft opening ticket sale limit...,4,3,1,2010,Universal Studios Singapore,100.000000,0.000000,0.000000,...,3.48,3.65,3.84,4.2,4.1,4.22,4.4,4.500,1.098,1.2


In [ ]:
# Checking the number of duplicated rows and rows with duplicate Review Texts.
duplicates = df[df.duplicated()]
review_text_duplicates = df[df.duplicated(subset=['Review_Text'])]
print("Number of entirely duplicated rows: " + str(duplicates.shape[0]))
print("Number of duplicate Review Texts: " + str(review_text_duplicates.shape[0]))

Number of entirely duplicated rows: 0
Number of duplicate Review Texts: 0


In [ ]:
# Check for null values
print(df.isnull().sum())

Surrogate Key                        0
Review_Text                          0
Rating                               0
Month                                0
Quarter                              0
Year                                 0
Branch                               0
Monthly % of Positive Reviews        0
Monthly % of Mixed Reviews           0
Monthly % of Negative Reviews        0
2006 Attendance (millions)       15731
2007 Attendance (millions)       15731
2008 Attendance (millions)       15731
2009 Attendance (millions)       15731
2010 Attendance (millions)           0
2011 Attendance (millions)           0
2012 Attendance (millions)           0
2013 Attendance (millions)           0
2014 Attendance (millions)           0
2015 Attendance (millions)           0
2016 Attendance (millions)           0
2017 Attendance (millions)           0
2018 Attendance (millions)           0
2019 Attendance (millions)           0
2020 Attendance (millions)           0
2021 Attendance (millions

# Load

In [ ]:
# Convert the dataframe to a csv with the delimiter being a , symbol.
# We do not include in the index in the csv.
df.to_csv('staged_data.csv', sep=',', index=False)

In [ ]:
# Download csv from Google Colab
files.download('staged_data.csv')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>